## Setting up vector retrieval
In order to retrieve top N most similar chunks based on a question, and evaluate retrieval performance

**Load previously generated Q and chunks**

In [22]:
from llama_index.finetuning import EmbeddingQAFinetuneDataset

# Load the datasets from saved json files
qa_dataset = EmbeddingQAFinetuneDataset.from_json(
    "data/qa_dataset_25-11-23_100_samples.json"
)

**Load the prev created llamaindex nodes**

In [21]:
from llama_index.storage.docstore import SimpleDocumentStore

# Load the SimpleDocumentStore from the persisted file
docstore = SimpleDocumentStore.from_persist_path("data/node_store_25-11-23")
# Create a list of nodes by list comprehension by iterating over the key (store hash) value (llamnaindex node) pairs of the docstore.docs object and appending the value to the list
nodes_vejledninger_filtered = [node for key, node in docstore.docs.items()]

In [23]:
nodes_vejledninger_filtered[0]

TextNode(id_='9281784d-734c-4665-b5dd-f6da8fd81859', embedding=None, metadata={'file_name': 'Vejledning om beskæftigelseskravet for ret til barselsdagpenge'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bf7cc5c5-d22b-4a6c-879b-39063472fd88', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_name': 'Vejledning om beskæftigelseskravet for ret til barselsdagpenge'}, hash='7e7f2f700c6c652f5cabbd0eef9eff850b88b2a95686f15eedec7b2391297a72'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='bbeb5fdc-a88b-4981-861c-65d14ff2d3ec', node_type=<ObjectType.TEXT: '1'>, metadata={'file_name': 'Vejledning om beskæftigelseskravet for ret til barselsdagpenge'}, hash='5c426770052433ab486974eab74a39cc111e1783c768f337dd2024f19cad06d7')}, hash='873272c4371dd3da03ab7d791cb036ba96a50b59f1f6b484639f3d6e9c868b1c', text='Vejledning om beskæftigelseskravet for ret til barselsdagpenge\n\n1.Indledning\nI denne vejl

**Set up embedding model for retrieval**

In [17]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model_hf = HuggingFaceEmbedding(
    model_name="intfloat/multilingual-e5-small",
    normalize=True,
    # Instruction to prepend query
    query_instruction="query:",
    # Instruction to prepend text
    text_instruction="passage:"
    # Or loading default values via
    # embed_model="local:intfloat/multilingual-e5-small"
)

In [18]:
from llama_index import ServiceContext

service_context_e5 = ServiceContext.from_defaults(embed_model=embed_model_hf)

In [24]:
from llama_index import VectorStoreIndex

# Building a vector index from the first 300 text cunks
vector_index_e5 = VectorStoreIndex(
    nodes_vejledninger_filtered[0:300],
    service_context=service_context_e5,
    show_progress=True,
)

Generating embeddings:   0%|          | 0/300 [00:00<?, ?it/s]

**Simple retriever test**

In [25]:
qa_dataset.query_docid_pairs[:2]

[('Hvad er de fem krav om tilknytning til arbejdsmarkedet, som en lønmodtager skal opfylde for at have ret til barselsdagpenge?',
  ['9281784d-734c-4665-b5dd-f6da8fd81859']),
 ('Hvad er betingelserne for at opfylde kravet om aktuel beskæftigelse og beskæftigelse i mindst 160 timer for at få ret til barselsdagpenge?',
  ['9281784d-734c-4665-b5dd-f6da8fd81859'])]

In [26]:
sample_query = qa_dataset.query_docid_pairs[0][0]
print(sample_query)

Hvad er de fem krav om tilknytning til arbejdsmarkedet, som en lønmodtager skal opfylde for at have ret til barselsdagpenge?


In [30]:
from llama_index.response.notebook_utils import display_source_node

retriever_e5 = vector_index_e5.as_retriever(similarity_top_k=3)
retrieved_nodes = retriever_e5.retrieve(sample_query)

for node in retrieved_nodes:
    display_source_node(node, source_length=1000)

**Node ID:** 3011dcf7-0ef2-43cb-9668-880ed01a5de5<br>**Similarity:** 0.969562923310707<br>**Text:** Det fremgår af lovens § 27, stk. 1, nr. 1, at en lønmodtager har ret til barselsdagpenge, hvis pågældende er i beskæftigelse dagen før fraværet eller på første fraværsdag og har været beskæftiget i mindst 160 timer inden for de seneste 4 afsluttede kalendermåneder og i mindst 3 af disse måneder har været beskæftiget i mindst 40 timer i hver måned. Kravet har til formål at sikre, at lønmodtagere, der opnår ret til barselsdagpenge, både har en aktuel tilknytning til arbejdsmarkedet og i en længere periode forud for første fraværsdag har haft en tilknytning til arbejdsmarkedet.<br>

**Node ID:** 490017fc-6fc4-4d52-a6ec-ca27ecd23761<br>**Similarity:** 0.9649478794124629<br>**Text:** Det betyder, at f.eks. en mor, der ikke opfylder beskæftigelseskravet på fødselstidspunktet, og dermed ikke får ret til de 10 ugers barselsdagpenge i den første fraværsperiode efter § 7, alligevel kan få ret til efterfølgende fravær efter § 9 med ret til barselsdagpenge efter § 21, hvis hun opfylder beskæftigelseskravet på det tidspunkt, hvor orloven efter § 9 påbegyndes.

Eksempel<br>

**Node ID:** a2609188-174a-4171-8a9a-c5655e92c248<br>**Similarity:** 0.9633721892746884<br>**Text:** Hvis en forælder inden for det første år har haft en periode uden barselsdagpenge og igen får ret til barselsdagpenge efter lovens § 29, stk. 1, 2 pkt., sker beregningen af barselsdagpengene ud fra den første fraværsdag i den nye barselsperiode. Hvis forælderen f.eks. ikke længere er i beskæftigelse, men nu ville have ret til arbejdsløshedsdagpenge, hvis forælderen ikke var på barsel, udbetales barselsdagpengene med den sats, personen kunne have fået i arbejdsløshedsdagpenge. Hvis forælderen f.eks. var selvstændig og er blevet lønmodtager, sker beregningen efter beregningsreglerne for lønmodtagere. For en lønmodtager, der har afbrudt en fraværsperiode, og som inden for en kort periode efter afbrydelsen, på ny påbegynder barselsfravær, beregnes barselsdagpengene dog på grundlag af samme timetal og timefortjeneste som i seneste forløb med barselsdagpenge, hvis lønmodtageren ikke har fået løn fra aktuelle arbejdsgiver under afbrydelsen, jf. § 9 i bekendtgørelsen om opgørelse af beskæft...<br>

**Conclusion: Intended match NOT in top 3** But perhaps the chunks still answer the question, possibly even better than the "correct" from the synthetic generation

## Evaluation (based on retriever, k=3)

In [31]:
from llama_index.evaluation import RetrieverEvaluator

retriever_evaluator_e5 = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever_e5
)

In [32]:
# try it out on an entire dataset, async evaluation to speed up
eval_results_e5 = await retriever_evaluator_e5.aevaluate_dataset(qa_dataset)

In [33]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"retrievers": [name], "hit_rate": [hit_rate], "mrr": [mrr]}
    )

    return metric_df

In [34]:
display_results("top-3 eval", eval_results_e5)

,retrievers,hit_rate,mrr
0,top-3 eval,0.805,0.683333
